# 2.2.2 チャットアプリ

In [ ]:
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage

load_dotenv("../.env")

In [ ]:
#1 モデルの定義
llm = ChatOpenAI(model="gpt-4o-mini")

history = []
n = 10
for i in range(10):
    user_input = input("ユーザ入力: ")
    if user_input == "exit":
        break
    #2 HumanMessage の作成と表示
    human_message = HumanMessage(user_input)
    human_message.pretty_print()
    #3 会話履歴の追加
    history.append(HumanMessage(user_input))
    #4 応答の作成と表示
    ai_message = llm.invoke(history)
    ai_message.pretty_print()
    #5 会話履歴の追加
    history.append(ai_message)

# 2.2.3 翻訳アプリ

In [ ]:
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

load_dotenv("../.env")

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")

#1 プロンプトテンプレートの作成
TRANSLATION_PROMPT = """\
以下の文章を {language} に翻訳し、翻訳結果のみを返してください。
{source_text}
"""
prompt = PromptTemplate.from_template(TRANSLATION_PROMPT)

#2 Runnable の作成
runnable = prompt | llm

language = "日本語"
source_text = """\
cogito, ergo sum
"""

#3 Runnable の実行と結果の表示
response = runnable.invoke(dict(language=language, source_text=source_text))
response.pretty_print()

# 2.2.4 テーブル作成アプリ

In [ ]:
from langchain.tools import tool
from langchain.pydantic_v1 import BaseModel, Field
from langchain_openai.chat_models import ChatOpenAI
import csv

#1 入力の定義
class CSVSaveToolInput(BaseModel):
    filename: str = Field(description="ファイル名")
    csv_text: str = Field(description="CSVのテキスト")

#2 ツールの定義
@tool("csv-save-tool", args_schema=CSVSaveToolInput)
def csv_save(filename: str, csv_text: str) -> bool:
    """CSV テキストをファイルに保存する"""
    # parse CSV text
    try:
        rows = list(csv.reader(csv_text.splitlines()))
    except Exception as e:
        return False

    # save to file
    with open(filename, "w") as f:
        writer = csv.writer(f)
        writer.writerows(rows)

    return True

In [ ]:
#1 ツールを LLM に紐づける
llm = ChatOpenAI(model="gpt-4o-mini")
tools = [csv_save]
llm_with_tool = llm.bind_tools(tools=tools)

TABLE_PROMPT = """\
{user_input}

結果は CSV ファイルに保存してください。ただし、ファイル名は上記の内容から適切に決定してください。
"""
prompt = PromptTemplate.from_template(TABLE_PROMPT)
get_tool_args = lambda x: x.tool_calls[0]["args"]

#2 Runnable の作成
runnable = prompt | llm_with_tool | get_tool_args | csv_save

user_input = "フィボナッチ数列の番号と値を10番目まで表にまとめて、CSV ファイルに保存してください。"

In [ ]:
response = runnable.invoke(dict(user_input=user_input))
print(response)

# Plan-and-Solve

In [ ]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv("../.env")
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
class ActionItem(BaseModel):
    action_name: str = Field(description="アクション名")
    action_description: str = Field(description="アクションの詳細")


class Plan(BaseModel):
    """アクションプランを格納する"""
    problem: str = Field(description="問題の説明")
    actions: list[ActionItem] = Field(description="実行すべきアクションリスト")


class ActionResult(BaseModel):
    """実行時の考えと結果を格納する"""

    thoughts: str = Field(description="検討内容")
    result: str = Field(description="結果")

In [ ]:
from langchain.output_parsers.openai_tools import PydanticToolsParser

ACTION_PROMPT = """\
問題をアクションプランに分解して解いています。
これまでのアクションの結果と、次に行うべきアクションを示すので、最新のアクションに関する情報のみを返してください。
# 問題
{problem}
# アクションプラン
{action_items}
# これまでのアクションの結果
{action_results}
# 次のアクション
{next_action}
"""

llm_action = llm.bind_tools([ActionResult], tool_choice="ActionResult")
action_parser = PydanticToolsParser(tools=[ActionResult], first_tool_only=True)
plan_parser = PydanticToolsParser(tools=[Plan], first_tool_only=True)

action_prompt = PromptTemplate.from_template(ACTION_PROMPT)
action_runnable = action_prompt | llm_action | action_parser

In [ ]:
from langchain_core.messages import AIMessage


def action_loop(action_plan: Plan):
    problem = action_plan.problem
    actions = action_plan.actions

    action_items = "\n".join(["* " + action.action_name for action in actions])
    action_results = []
    action_results_str = ""
    for i, action in enumerate(actions):
        print("="*20)
        print(f"[{i+1}/{len(actions)}]以下のアクションを実行します。")
        print(action.action_name)

        next_action = f"* {action.action_name}  \n{action.action_description}"
        response = action_runnable.invoke(dict(problem=problem, action_items=action_items, action_results=action_results_str, next_action=next_action))
        action_results.append(response)
        action_results_str += f"* {action.action_name}  \n{response.result}\n" 
        print("-" *10 + "検討内容" + "-"*10)
        print(response.thoughts)
        print("-" *10 + "結果" + "-" * 10)
        print(response.result)

    return AIMessage(action_results_str)

In [ ]:
def route(ai_message: AIMessage):
    if ai_message.response_metadata["finish_reason"] == "tool_calls":
        return plan_parser | RunnableLambda(action_loop)
    else:
        return ai_message

In [110]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage

PLAN_AND_SOLVE_PROMPT = f"""\
ユーザの質問が複雑な場合は、アクションプランを作成し、その後に1つずつ実行する Plan-and-Solve 形式をとります。
これが必要と判断した場合は、Plan ツールによってアクションプランを保存してください。
"""
system_prompt = SystemMessage(PLAN_AND_SOLVE_PROMPT)
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, MessagesPlaceholder(variable_name="history")])

llm_plan = llm.bind_tools(tools=[Plan])
planning_runnable = chat_prompt | llm_plan | route

In [111]:
history = []
n = 10
for i in range(10):
    user_input = input("ユーザ入力: ")
    if user_input == "exit":
        break
    #2 HumanMessage の作成と表示
    human_message = HumanMessage(user_input)
    human_message.pretty_print()
    #3 会話履歴の追加
    history.append(HumanMessage(user_input))
    #4 応答の作成と表示
    ai_message = planning_runnable.invoke(dict(history=history))
    ai_message.pretty_print()
    #5 会話履歴の追加
    history.append(ai_message)

================================ Human Message =================================

こんにちは
================================== Ai Message ==================================

こんにちは！今日はどのようなことをお手伝いできますか？
================================ Human Message =================================

f(x) = x^2 + x + 1 の極値を計算して
[1/3]以下のアクションを実行します。
関数の導関数を求める
----------検討内容----------
関数 f(x) = x^2 + x + 1 の導関数 f'(x) を求めるためには、微分を行う必要がある。微分を行うと、f'(x) = 2x + 1 となる。
----------結果----------
f'(x) = 2x + 1
[2/3]以下のアクションを実行します。
導関数をゼロに設定する
----------検討内容----------
導関数 f'(x) = 2x + 1 をゼロに設定し、x の値を求めることで、極値の候補を見つける。
----------結果----------
2x + 1 = 0 から x = -1/2 が得られた。
[3/3]以下のアクションを実行します。
極値を計算する
----------検討内容----------
得られた x の値 x = -1/2 を f(x) に代入して、極値を計算する。具体的には、f(-1/2) を求める。
----------結果----------
f(-1/2) = (-1/2)^2 + (-1/2) + 1 = 1/4 - 1/2 + 1 = 1/4 - 2/4 + 4/4 = 3/4 よって、極値は 3/4 である。
================================== Ai Message ==================================

* 関数の導関数を求める  
f'(x) = 2x + 1* 導関数をゼロに設定する  
2x